In [ ]:
from typing import Optional
from pydantic import Field, PrivateAttr
from langchain.tools import BaseTool
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.callbacks.manager import CallbackManagerForToolRun
from langchain.tools import tool

from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader


from langchain_core.vectorstores import InMemoryVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter


from typing import Optional
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain_core.callbacks import CallbackManagerForToolRun

class PDFTextExtractorTool(BaseTool):
    name: str = "PDFTextExtractor"
    description: str = "Extracts relevant data from a PDF file located at the specified path. Provide the path and query."

    def _run(self, query: str, pdf_path: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
            # Load the PDF file
            loader = PyPDFLoader(pdf_path)
            docs = loader.load()
            text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500, chunk_overlap=200, add_start_index=True)
            all_splits = text_splitter.split_documents(docs)
            vector_store = InMemoryVectorStore(embeddings)
            ids = vector_store.add_documents(documents=all_splits)
            results = vector_store.similarity_search(query)
            print(results[0])
            return results[0]


extract_tool = PDFTextExtractorTool()


from langchain_community.tools import TavilySearchResults

tavily_tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
)

from langchain.agents import AgentType, initialize_agent
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain_openai import ChatOpenAI


import os
from langchain_community.utilities import SerpAPIWrapper
os.environ["SERPAPI_API_KEY"] = "f9bec0683ce3cf7380f98d08f08e2b55666b4e37b13f074f663f7f154c480443"

@tool
def google_search(query: str):
    """Search Google for recent and realtime results."""
    search = SerpAPIWrapper()
    return search.run(query)


import os
import requests



def download_pdfs(urls: list, folder_path: str) -> None:
    """
    Downloads PDFs from a given list of URLs and saves them to the specified folder.

    :param urls: List of URLs of the PDF files.
    :param folder_path: The folder where the PDFs should be saved.
    """
    # Ensure the folder exists
    os.makedirs(folder_path, exist_ok=True)

    for index, url in enumerate(urls, start=1):
        try:
            # Extract the filename from the URL or use a default name
            filename = url.split('/')[-1]
            if not filename.endswith('.pdf'):
                filename = f"file_{index}.pdf"  # Fallback filename

            save_path = os.path.join(folder_path, filename)

            # Download and save the PDF
            response = requests.get(url)
            response.raise_for_status()  # Raise an error for HTTP issues
            with open(save_path, 'wb') as file:
                file.write(response.content)
            print(f"PDF successfully downloaded and saved to {save_path}")

        except Exception as e:
            print(f"Failed to download {url}: {e}")


folder_path = "/content/sample_data/pdfs"


from googlesearch import search


@tool
def downloader_tool(query: str, num_results: int = 3):
    """Downloads the online documents from across the internet.

    Args:
        query: The search query.
        num_results: The maximum number of search results to consider (default: 10).
    """
    # 1. Perform a web search for relevant PDFs (limit to top 10)
    search_results = search(f"{query} filetype:pdf", num=num_results)

    # Convert the generator to a list and return the top results
    urls = [url for _, url in zip(range(num_results), search_results)]

    download_pdfs(urls, folder_path)

    print("---HHAAAAA")



@tool
def online_retriever_tool():
    """Does nothing.A dummy tool that does nothing but satisfies the tool requirement."""
    return " "

@tool
def iterative_consolidation_tool(initial_context: str, max_iterations: int = 2) -> str:
    """Tool to get consolidated information from initial context."""
    consolidation_chain = LLMChain(llm=llm, prompt=consolidation_prompt)
    context = initial_context

    # Debug: Print the initial context received
    print("------------------")
    # print("\n Initial Context Passed to Tool:\n", context)

    for i in range(max_iterations):
        # Run iterative consolidation
        context = consolidation_chain.run({"context": context})

        # Debug: Print context after each iteration
        print(f" Iteration {i + 1} Context:\n", context)
        print("------------------")

    return context


@tool
def NoOpTool():
    """Does nothing.A dummy tool that does nothing but satisfies the tool requirement."""
    return " "



from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain.tools import Tool

class FinancialAbbreviationTool:
    def __init__(self, document_path: str):
        self.document_path = document_path
        self.db = self._load_documents()

    def _load_documents(self):
        raw_documents = TextLoader(self.document_path).load()
        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
        documents = text_splitter.split_documents(raw_documents)
        return FAISS.from_documents(documents, embeddings)

    def retrieve_full_form(self, abbreviation: str) -> str:
        results = self.db.similarity_search(abbreviation)
        if results:
            return results[0].page_content  # Return the most relevant match
        return f"Full form for '{abbreviation}' not found."

abbreviation_tool = FinancialAbbreviationTool("/content/sample_data/Jargon_collection.txt")

retrieve_abbreviation_tool = Tool(
    name="RetrieveFullForms",
    func=abbreviation_tool.retrieve_full_form,
    description="Use this tool to retrieve the full form of abbreviations or shorthand terms in financial context."
)




@tool
def generate_subquestions(query: str,file_path : str):
    """
    Tool to handle queries that require a complex retrieval.
    It can answer any question from simple to complex.

    """

    global final_sub_responses





    file_path=file_path
    # Define the system prompt dynamically based on the number of sub-questions
    system = f"""You are a helpful assistant designed to generate actionable sub-questions related to a given input query. Your primary goal is to break down complex queries into smaller, focused, and answerable sub-questions that can be independently addressed by LLMs or other tools.

### Instructions:
1. **Purpose**: Ensure each sub-question is specific and independently answerable to extract relevant information. The goal is to collect all necessary pieces of information for answering the user's main query.

2. **Relevance and Focus**:
   - Sub-questions must directly relate to the user's input query.
   - Avoid asking the user for additional information. Instead, focus on generating questions that the LLM or tools can answer using external data or documents.
   - Sub-questions should cover various aspects of the input query comprehensively but concisely.

3. **Avoid Asking Back**: Do not create sub-questions that ask the user for clarification or additional details. For example:
   - **Avoid**: "What specific financial metrics are you interested in?"
   - **Instead**: "What were Nykaa's total revenues in the last quarter of 2023?"

4. **Examples**:
   - Input Query: "Nykaa's last quarter financial report."
     - Sub-Questions:
       - "What was Nykaa's revenue in the last quarter of 2023?"
       - "What were Nykaa's expenses in the last quarter of 2023?"
       - "What was Nykaa's profit in the last quarter of 2023?"
       - "What is the overall performance of Nykaa in the last quarter of 2023?"

   - Input Query: "Impact of AI on healthcare."
     - Sub-Questions:
       - "What are the recent advancements in AI for disease detection?"
       - "How is AI used in personalized medicine?"
       - "What is the role of AI in drug discovery?"
       - "What are the benefits of AI in robotic surgeries?"
       - "What are the risks of using AI in healthcare?"

### Output Format:
- Generate sub-questions in a list format.
- Ensure the sub-questions are clear, concise, and directly actionable.
- Do not exceed 5 sub-questions.

"""

    # Initialize the structured model (example: a model capable of sub-question generation)
    structured_model = llm.with_structured_output(Questions)

    # Generate the sub-questions
    response = structured_model.invoke(
        [SystemMessage(content=system), HumanMessage(content=query)]
    )
